First import pantheon and asyncio, and initiate a Pantheon instance

In [1]:
from pantheon import pantheon
import asyncio

server = "euw1"
api_key = "RGAPI-XXXX"

panth = pantheon.Pantheon(server, api_key, True)

Create the function to get game data

In [2]:

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":10})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.getMatch(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)

Get you games data

In [3]:
name = "Canisback"

loop = asyncio.get_event_loop()  

(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))
gamesDataRaw = loop.run_until_complete(getRecentMatches(accountId))
print(gamesDataRaw[0])

{'participants': [{'spell1Id': 11, 'championId': 11, 'highestAchievedSeasonTier': 'UNRANKED', 'timeline': {'role': 'NONE', 'damageTakenPerMinDeltas': {'0-10': 686.4, '10-20': 841.2}, 'creepsPerMinDeltas': {'0-10': 1.3, '10-20': 3.8}, 'xpPerMinDeltas': {'0-10': 399, '10-20': 520.6}, 'damageTakenDiffPerMinDeltas': {'0-10': 103.80000000000001, '10-20': -93.00000000000006}, 'lane': 'JUNGLE', 'goldPerMinDeltas': {'0-10': 403.4, '10-20': 581.0999999999999}, 'participantId': 1, 'csDiffPerMinDeltas': {'0-10': 1.3, '10-20': 2.6}, 'xpDiffPerMinDeltas': {'0-10': 123.89999999999999, '10-20': 224.10000000000002}}, 'teamId': 100, 'participantId': 1, 'spell2Id': 4, 'stats': {'neutralMinionsKilledTeamJungle': 45, 'magicDamageDealtToChampions': 82, 'item3': 3006, 'firstTowerAssist': False, 'perk2Var3': 0, 'unrealKills': 0, 'perk5Var2': 0, 'trueDamageTaken': 188, 'win': True, 'perk3': 8014, 'perk5Var1': 14, 'sightWardsBoughtInGame': 0, 'longestTimeSpentLiving': 401, 'firstInhibitorKill': True, 'timeCCin

The first step in this analysis is to figure out what is the participantId of the targeted player.

In [4]:
def getParticipantIdFromAccountId(game,accountId):
    for participant in game['participantIdentities']:
        if participant['player']['currentAccountId'] == accountId:
            return participant['participantId']
        
participantId = getParticipantIdFromAccountId(gamesDataRaw[0],accountId)
participantId

5

Now our targeted data are champion played by the player, and team composition. First, we get to now what champion has our player and in which team he is, and if he won.

In [5]:
def getTeamAndChampion(game, participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            return participant['teamId'],participant['championId'],participant['stats']['win']

teamId, championId, win = getTeamAndChampion(gamesDataRaw[0], participantId)
print(teamId, championId, win)

100 89 True


A little trick to know for teamId is that any summoner with participantId from 1 to 5 is in team 100 and from 6 to 10 in team 200, but it's only for 5v5 games.

Next is getting team composition.

In [6]:
def getTeamComposition(game):
    #Create an array for each team
    compositions = {100:[],200:[]}
    for participant in game['participants']:
        compositions[participant['teamId']].append(participant['championId'])
    return compositions

compositions = getTeamComposition(gamesDataRaw[0])
compositions

{100: [11, 110, 90, 31, 89], 200: [63, 150, 222, 86, 24]}

Now that we have all team compositions, we would want to have their name instead of championId. Let's use ddragon to achieve that by creating a championId=>name dictionnary

In [7]:
import requests, json
response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.9.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championIdToName = {}
for key,champion in champonRawData['data'].items():
    championIdToName[int(champion['key'])] = champion['name']
championIdToName

{1: 'Annie',
 2: 'Olaf',
 3: 'Galio',
 4: 'Twisted Fate',
 5: 'Xin Zhao',
 6: 'Urgot',
 7: 'LeBlanc',
 8: 'Vladimir',
 9: 'Fiddlesticks',
 10: 'Kayle',
 11: 'Master Yi',
 12: 'Alistar',
 13: 'Ryze',
 14: 'Sion',
 15: 'Sivir',
 16: 'Soraka',
 17: 'Teemo',
 18: 'Tristana',
 19: 'Warwick',
 20: 'Nunu',
 21: 'Miss Fortune',
 22: 'Ashe',
 23: 'Tryndamere',
 24: 'Jax',
 25: 'Morgana',
 26: 'Zilean',
 27: 'Singed',
 28: 'Evelynn',
 29: 'Twitch',
 30: 'Karthus',
 31: "Cho'Gath",
 32: 'Amumu',
 33: 'Rammus',
 34: 'Anivia',
 35: 'Shaco',
 36: 'Dr. Mundo',
 37: 'Sona',
 38: 'Kassadin',
 39: 'Irelia',
 40: 'Janna',
 41: 'Gangplank',
 42: 'Corki',
 43: 'Karma',
 44: 'Taric',
 45: 'Veigar',
 48: 'Trundle',
 50: 'Swain',
 51: 'Caitlyn',
 53: 'Blitzcrank',
 54: 'Malphite',
 55: 'Katarina',
 56: 'Nocturne',
 57: 'Maokai',
 58: 'Renekton',
 59: 'Jarvan IV',
 60: 'Elise',
 61: 'Orianna',
 62: 'Wukong',
 63: 'Brand',
 64: 'Lee Sin',
 67: 'Vayne',
 68: 'Rumble',
 69: 'Cassiopeia',
 72: 'Skarner',
 74: 'Hei

In [8]:
playerChampion = championIdToName[championId]
playerChampion

'Leona'

In [9]:
teamComposition = [championIdToName[chId] for chId in compositions[teamId]]
teamComposition

['Master Yi', 'Varus', 'Malzahar', "Cho'Gath", 'Leona']

In [10]:
enemyTeamComposition = [championIdToName[chId] for chId in compositions[100 if teamId==200 else 200]]
enemyTeamComposition

['Brand', 'Gnar', 'Jinx', 'Garen', 'Jax']

You can also transform the array in a single string

In [11]:
"/".join(teamComposition)

"Master Yi/Varus/Malzahar/Cho'Gath/Leona"

In [12]:
"/".join(enemyTeamComposition)

'Brand/Gnar/Jinx/Garen/Jax'

Next step is getting a list of items the player had. In the same way we get champion names, we need items name too.

In [13]:
response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.9.1/data/en_US/item.json')
itemRawData = json.loads(response.text)

itemIdToName = {}
for key,item in itemRawData['data'].items():
    itemIdToName[int(key)] = item['name']

def getItemList(game, participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            itemsList = []
            for i in range(0,7):
                if not participant['stats']['item'+str(i)] == 0:
                    itemsList.append(itemIdToName[participant['stats']['item'+str(i)]])
            return " / ".join(itemsList)
getItemList(gamesDataRaw[0], participantId)

"Broken Stopwatch / Remnant of the Aspect / Shurelya's Reverie / Iceborn Gauntlet / Boots of Swiftness / Null-Magic Mantle / Oracle Lens"

Add some other data like patch and date

In [14]:
def getPatch(game):
    #Game version number contains minor versions, only take the two major ones.
    return ".".join(game['gameVersion'].split(".")[:2])
getPatch(gamesDataRaw[0])

'8.9'

In [15]:
import datetime

def getDate(game):
    return datetime.datetime.fromtimestamp(int(game["gameCreation"]/1000)).strftime('%Y/%m/%d')

getDate(gamesDataRaw[0])

'2018/05/12'

Now that we have all the data we need (for now), let's generalize it for every game we get

In [16]:
gameInformationList = []

for game in gamesDataRaw:
    #Get the information
    participantId = getParticipantIdFromAccountId(game,accountId)
    teamId, championId, win = getTeamAndChampion(game, participantId)
    compositions = getTeamComposition(game)
    
    #Select information you want to show
    gameInformation = {}
    gameInformation['win'] = win
    gameInformation['champion'] = championIdToName[championId]
    gameInformation['patch'] = getPatch(game)
    gameInformation['date'] = getDate(game)
    gameInformation['teamComposition'] = "/".join([championIdToName[chId] for chId in compositions[teamId]])
    gameInformation['enemyTeamComposition'] = "/".join([championIdToName[chId] for chId in compositions[100 if teamId==200 else 200]])
    #Add the row to the list
    gameInformationList.append(gameInformation)

gameInformationList

[{'champion': 'Leona',
  'date': '2018/05/12',
  'enemyTeamComposition': 'Brand/Gnar/Jinx/Garen/Jax',
  'patch': '8.9',
  'teamComposition': "Master Yi/Varus/Malzahar/Cho'Gath/Leona",
  'win': True},
 {'champion': 'Leona',
  'date': '2018/05/12',
  'enemyTeamComposition': 'Yorick/Janna/Jarvan IV/Miss Fortune/Riven',
  'patch': '8.9',
  'teamComposition': 'Varus/Lux/Trundle/Leona/Tryndamere',
  'win': False},
 {'champion': 'Leona',
  'date': '2018/05/12',
  'enemyTeamComposition': 'Jinx/Janna/Zed/Shyvana/Jax',
  'patch': '8.9',
  'teamComposition': 'Irelia/Varus/Warwick/Veigar/Leona',
  'win': True},
 {'champion': 'Leona',
  'date': '2018/05/12',
  'enemyTeamComposition': 'Garen/Shen/Vi/LeBlanc/Graves',
  'patch': '8.9',
  'teamComposition': 'Yasuo/Varus/Warwick/Veigar/Leona',
  'win': True},
 {'champion': 'Leona',
  'date': '2018/05/12',
  'enemyTeamComposition': 'Vladimir/Kennen/Master Yi/Lux/Nidalee',
  'patch': '8.9',
  'teamComposition': 'Varus/Warwick/Jax/Leona/Veigar',
  'win': T

Great, you have your data! I recommend you to use pandas to display nd manipulate the data easily

In [17]:
import pandas as pd
df = pd.DataFrame(gameInformationList)
df

,champion,date,enemyTeamComposition,patch,teamComposition,win
0,Leona,2018/05/12,Brand/Gnar/Jinx/Garen/Jax,8.9,Master Yi/Varus/Malzahar/Cho'Gath/Leona,True
1,Leona,2018/05/12,Yorick/Janna/Jarvan IV/Miss Fortune/Riven,8.9,Varus/Lux/Trundle/Leona/Tryndamere,False
2,Leona,2018/05/12,Jinx/Janna/Zed/Shyvana/Jax,8.9,Irelia/Varus/Warwick/Veigar/Leona,True
3,Leona,2018/05/12,Garen/Shen/Vi/LeBlanc/Graves,8.9,Yasuo/Varus/Warwick/Veigar/Leona,True
4,Leona,2018/05/12,Vladimir/Kennen/Master Yi/Lux/Nidalee,8.9,Varus/Warwick/Jax/Leona/Veigar,True
5,Leona,2018/05/12,Nunu/Xin Zhao/Caitlyn/Thresh/Jax,8.9,Yasuo/Varus/Nasus/Jax/Leona,True
6,Malzahar,2018/05/12,Ryze/Dr. Mundo/Varus/Blitzcrank/Lee Sin,8.9,Ezreal/Graves/Darius/Blitzcrank/Malzahar,False
7,Ezreal,2018/05/12,Karma/Varus/Amumu/Renekton/Zoe,8.9,Vel'Koz/Kassadin/Darius/Jarvan IV/Ezreal,False
8,Caitlyn,2018/05/12,Jayce/Vayne/Alistar/Master Yi/Kassadin,8.9,Warwick/Cho'Gath/Brand/Garen/Caitlyn,False
9,Annie,2018/05/12,Warwick/Thresh/Wukong/Blitzcrank/Fizz,8.9,Xin Zhao/Jhin/Darius/Brand/Annie,False
